In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("QuickStart").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/03/29 00:24:43 WARN Utils: Your hostname, dev resolves to a loopback address: 127.0.1.1; using 172.27.194.33 instead (on interface eth0)
25/03/29 00:24:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/29 00:24:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 58864)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/arek/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/arek/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/arek/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 271, in accum_updates
    nu

In [5]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1,1,12,0)),
    Row(a=2, b=3., c='string2', d=date(2001, 2, 2), e=datetime(2001, 1,2,12,0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2002, 1,3,12,0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [6]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [7]:
from datetime import date, datetime
import pandas as pd
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
print(pandas_df.dtypes)

pandas_df['d'] = pd.to_datetime(pandas_df['d'])
print(pandas_df.dtypes)
df = spark.createDataFrame(pandas_df)
df

a             int64
b           float64
c            object
d            object
e    datetime64[ns]
dtype: object
a             int64
b           float64
c            object
d    datetime64[ns]
e    datetime64[ns]
dtype: object


DataFrame[a: bigint, b: double, c: string, d: timestamp, e: timestamp]

In [8]:
df.show()

+---+---+-------+-------------------+-------------------+
|  a|  b|      c|                  d|                  e|
+---+---+-------+-------------------+-------------------+
|  1|2.0|string1|2000-01-01 00:00:00|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01 00:00:00|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01 00:00:00|2000-01-03 12:00:00|
+---+---+-------+-------------------+-------------------+



In [9]:
print(df.count())
print(df.columns)
print(df.schema)



3
['a', 'b', 'c', 'd', 'e']
StructType([StructField('a', LongType(), True), StructField('b', DoubleType(), True), StructField('c', StringType(), True), StructField('d', TimestampType(), True), StructField('e', TimestampType(), True)])


In [10]:
df.show(1)

+---+---+-------+-------------------+-------------------+
|  a|  b|      c|                  d|                  e|
+---+---+-------+-------------------+-------------------+
|  1|2.0|string1|2000-01-01 00:00:00|2000-01-01 12:00:00|
+---+---+-------+-------------------+-------------------+
only showing top 1 row



In [11]:
spark.conf.set('spark.sql.repl.eagerEval.enabled',True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01 00:00:00,2000-01-01 12:00:00
2,3.0,string2,2000-02-01 00:00:00,2000-01-02 12:00:00
3,4.0,string3,2000-03-01 00:00:00,2000-01-03 12:00:00


In [12]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2.0                 
 c   | string1             
 d   | 2000-01-01 00:00:00 
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [13]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [14]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: timestamp (nullable = true)
 |-- e: timestamp (nullable = true)



In [15]:
df.select("a","b","c").describe().show()

+-------+---+---+-------+
|summary|  a|  b|      c|
+-------+---+---+-------+
|  count|  3|  3|      3|
|   mean|2.0|3.0|   NULL|
| stddev|1.0|1.0|   NULL|
|    min|  1|2.0|string1|
|    max|  3|4.0|string3|
+-------+---+---+-------+



In [16]:
df.collect()

[Row(a=1, b=2.0, c='string1', d=datetime.datetime(2000, 1, 1, 0, 0), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3.0, c='string2', d=datetime.datetime(2000, 2, 1, 0, 0), e=datetime.datetime(2000, 1, 2, 12, 0)),
 Row(a=3, b=4.0, c='string3', d=datetime.datetime(2000, 3, 1, 0, 0), e=datetime.datetime(2000, 1, 3, 12, 0))]

In [17]:
df.take(1)

[Row(a=1, b=2.0, c='string1', d=datetime.datetime(2000, 1, 1, 0, 0), e=datetime.datetime(2000, 1, 1, 12, 0))]

In [18]:
df.toPandas()

,a,b,c,d,e
0,1,2.0,string1,2000-01-01,2000-01-01 12:00:00
1,2,3.0,string2,2000-02-01,2000-01-02 12:00:00
2,3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [19]:
df.a

Column<'a'>

In [20]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [21]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

type(df.c) == type(upper(df.c)) == type(df.c.isNull)

False

In [22]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [23]:
df.withColumn('upper_c', upper(df.c)).show()

+---+---+-------+-------------------+-------------------+-------+
|  a|  b|      c|                  d|                  e|upper_c|
+---+---+-------+-------------------+-------------------+-------+
|  1|2.0|string1|2000-01-01 00:00:00|2000-01-01 12:00:00|STRING1|
|  2|3.0|string2|2000-02-01 00:00:00|2000-01-02 12:00:00|STRING2|
|  3|4.0|string3|2000-03-01 00:00:00|2000-01-03 12:00:00|STRING3|
+---+---+-------+-------------------+-------------------+-------+



In [24]:
df.filter(df.a == 1).show()

+---+---+-------+-------------------+-------------------+
|  a|  b|      c|                  d|                  e|
+---+---+-------+-------------------+-------------------+
|  1|2.0|string1|2000-01-01 00:00:00|2000-01-01 12:00:00|
+---+---+-------+-------------------+-------------------+



In [25]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 2|
|                 3|
|                 4|
+------------------+



In [26]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

+---+---+-------+-------------------+-------------------+
|  a|  b|      c|                  d|                  e|
+---+---+-------+-------------------+-------------------+
|  1|2.0|string1|2000-01-01 00:00:00|2000-01-01 12:00:00|
+---+---+-------+-------------------+-------------------+



In [27]:
df = spark.createDataFrame([
    ['red','banana', 1, 10], ['blue','banana', 2,20], ['red','carrot', 3,30],
    ['blue','grape', 4,40], ['red','carrot', 5,50], ['black','carrot', 6,60],
    ['red','banana', 7,70], ['red','grape', 8,80]], schema=['color','fruit','v1','v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [28]:
df.groupby('color').avg().show

<bound method DataFrame.show of +-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+
>

In [29]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  0| 60|
| blue|banana| -1| 20|
| blue| grape|  1| 40|
|  red|banana| -3| 10|
|  red|carrot| -1| 30|
|  red|carrot|  0| 50|
|  red|banana|  2| 70|
|  red| grape|  3| 80|
+-----+------+---+---+



In [30]:
df.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6| 60|
| blue|banana|  2| 20|
|  red|banana|  1| 10|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [33]:
df.createOrReplaceTempView("tableA")
spark.sql("select count(*) from tableA").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [37]:
spark.sql("select * from tableA").show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [38]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s+1

spark.udf.register("add_one", add_one)
spark.sql("select add_one(v1) from tableA").show()


25/03/29 00:35:09 WARN SimpleFunctionRegistry: The function add_one replaced a previously registered function.


+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



In [39]:
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+

+--------------+
|(count(1) > 0)|
+--------------+
|          true|
+--------------+



25/03/29 12:32:17 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 10152206 ms exceeds timeout 120000 ms
25/03/29 12:32:17 WARN SparkContext: Killing executors is not supported by current scheduler.
25/03/29 12:32:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint